

## 什么是命名实体

命名实体(named entity)，是人名、机构名、地名等以名称为标识的，有特定意义的实体，如“济南”、“春晚”，“张杰”

![Markdown](http://i2.tiimg.com/611786/76d39a0555e6b56e.png)

NER研究的命名实体一般分为3大类、7小类，其中，时间、日期、货币等实体识别格式比较固定，能采用模式匹配的方式识别，人名、地名、机构名较为复杂，所以近年对NER的研究主要是针对这几种实体的。

+ 三大类：实体类、时间类以及数字类
+ 七小类：人名、地名、组织机构名、时间、日期、货币和百分比

## 什么是命名实体识别

识别出文本中所需的命名实体，并把识别出词在文本序列中标注出来的过程，叫作命名实体识别（Named Entity Recognition），简称NER，是信息检索、问答系统等技术的基础任务。

## 进行命名实体识别的方法

+ 基于规则的命名实体识别

依赖于手工规则的系统，结合命名实体库，对每条规则进行权重赋值，然后通过实体与规则的匹配情况进行判断。但大多数情况下，规则依赖与具体的语言、领域和文本风格，难以涵盖所有的语言现象。

+ 基于统计的命名实体识别

基于人工标注的语料，将命名实体识别任务作为序列标注问题来解决。这种方法对语料库的依赖比较大，语料库的选取会影响识别结果，而且可以用于建设和评估识别系统的大规模通用语料库很少。

+ 混合方法

大多数情况下，要先借助规则知识提前进行过滤修剪处理，再使用基于统计的混合方法。


## 基于条件随机场的命名实体识别

**为什么使用条件随机场？**

如果有Justin Bieber一天的生活快照，你想在这些照片上打上活动内容的标签（吃饭、睡觉、开车、演出），需要怎么做？

可以忽略这些照片的本质，建立一个图片分类器，比如事先给定一个月的打码快照，通过观察这些打码快照，学习到在早上5点拍的光线很暗的照片，很可能是在睡觉，颜色很明亮有打光的照片，可能是在演出唱歌，照片里有车出现的可能是在开车等等。

再看这些照片的顺序，如果你看到一张嘴张开的特写照片，可能在吃饭也可能是在唱歌，这样就无法确认，但这张特写的上一张照片里有做饭或是准备餐具的照片，就能这张特写照极大可能是在吃饭了。

因此，参考相近的照片，可以提高我们对当前照片打码的准确率，这就是条件随机场在做的事情。

### 数据预处理

将语料库

![Markdown](http://i2.tiimg.com/611786/ec1ac03d469a12bc.png)

处理为可识别的文本格式：

![Markdown](http://i2.tiimg.com/611786/e68b59aebd57ac01.png)

将处理后的文本分为训练集和测试集（80%为训练集，20%为测试集）

### 特征的选取

CRF模型识别的性能在很大程度上依赖于特征模板的设计，而设计特征模板的难点在于特征的选择及特征组合方式，以及单一特征或特征组合所设定的窗口长度。

**窗口：以当前单词或字为中心的上下文，窗口长度越大，就能结合更多的上下文信息。但窗口长度过大会引起过拟合，窗口长度过小会导致信息丢失**

不同的语言模型和不同的任务目标，特征模板各有不同。模板中的特征选择越多，模型在训练和标注时的数据量就越大，耗费的时间越长，对机器资源要求更高。

特征模板可以是单一特征，也可以是特征组合，选定好窗口大小后，进行实验，找出$F_1$最大的特征模板：

![Markdown](http://i2.tiimg.com/611786/3c24c0caf356393a.png)

![Markdown](http://i2.tiimg.com/611786/ec726c4fd55335ac.png)

![Markdown](http://i2.tiimg.com/611786/3bc8fdaa327aecb7.png)

### 特征模板

![Markdown](http://i2.tiimg.com/611786/2f17c131ee3fe123.png)



以人民日报语料库为例：

![Markdown](http://i1.fuimg.com/611786/84929b4547a3e4ad.png)

#### Unigram模板

当遍历到“网”这行时，“网”即为当前行，模板举例如下：

![Markdown](http://i1.fuimg.com/611786/508c020747022a83.png)

![Markdown](http://i1.fuimg.com/611786/ccdbf5d3aecfd7bb.png)

如%X[0，0]用特征函数打分的形式，表现如下：

func1 = if (output = S and feature="U00:网") return 1 else return 0
func2 = if (output = B and feature="U00:网") return 1 else return 0
func3 = if (output = M and feature="U00:网") return 1 else return 0
func4 = if (output = E and feature="U00:网") return 1 else return 0

#### Bigram模板

Bigram并没有直接定义模板函数，只写了一个B，默认产生当前状态与下一状态间的组合，更好地利用上下文间的关系,即四个标签B、M、E、S相互间的关系，例如，B后面有多大可能性接M，E后面多大可能性接B，类似于HMM中的转移矩阵，是隐藏状态间的转移。

还是“人民网”举例，将产生下列的特征函数：y1是前一个状态，y2是后一个状态。

If(y1==’B’&&y2==’B’) return 1 else return 0; **当然这个特征函数应该是不合理的，当前字符是Begin的状态，下一个只能是Middle或者End。**

func1 = If(y1==’B’&&y2==’B’) return 1 else return 0;

func1 = If(y1==’B’&&y2==’M’) return 1 else return 0;

func1 = If(y1==’B’&&y2==’E’) return 1 else return 0;

func1 = If(y1==’B’&&y2==’S’) return 1 else return 0;

func1 = If(y1==’M’&&y2==’B’) return 1 else return 0;

func1 = If(y1==’M’&&y2==’M’) return 1 else return 0;

func1 = If(y1==’M’&&y2==’E’) return 1 else return 0;

func1 = If(y1==’M’&&y2==’S’) return 1 else return 0;

func1 = If(y1==’E’&&y2==’B’) return 1 else return 0;

func1 = If(y1==’E’&&y2==’M’) return 1 else return 0;

func1 = If(y1==’E’&&y2==’E’) return 1 else return 0;

func1 = If(y1==’E’&&y2==’S’) return 1 else return 0;

func1 = If(y1==’S’&&y2==’B’) return 1 else return 0;

func1 = If(y1==’S’&&y2==’M’) return 1 else return 0;

func1 = If(y1==’S’&&y2==’E’) return 1 else return 0;

func1 = If(y1==’S’&&y2==’S’) return 1 else return 0;


#### 参数学习，训练模型

对于给定的训练集{$(x^t,y^t)$}(t=1,..,N)，需要找到一个最合适的参数使得在x（句子）的条件下y（标注）的后验概率最大，即求解$\hat{\theta} = arg max \prod_{i=1}^NP(y^t|x^t)$

![Markdown](http://i1.fuimg.com/611786/32bbb4f790de33ea.png)

参数$\hat{\theta}$在CRF中是两个特征函数的权重，$\lambda$、$\mu$分别表示Unigram模板和Bigram模板产生的特征函数的权重。

在标注过程中，如何知道哪种标注序列是最好的？我们采用评分制，分数越高的标注得越好，评分规则就是我们下面要介绍的特征函数，就像设置了一个Loss Function = score（所有标注序列加和）- score（真实的标注序列），我们在寻找一个最合适的参数$\lambda$和$\mu$，使得真实标注序列的得分最高，其他不是真实的标注序列的得分尽可能地低，才能令损失最小，方法就是参数能够尽量提高正确的规则（即特征函数）的分数，降低错误的规则的分数，使得有用的规则更有利于最佳路径，参数学习的迭代过程就是这样一个不断降低损失直到损失最低的过程。


##### 整体思路

Loss Function = Score（所有标注序列加和）- Score（最佳的标注序列） = $Score_{all}$ - $Score_{real}$

$Score_{real}$ = $\sum\lambda f_A$ + $\sum\mu f_B$

其中，$f_A$即为func1 = if (output = S and feature="U00:网") return 1 else return 0中return的值

$f_B$即为If(y1==’B’&&y2==’B’) return 1 else return 0 中return的值


##### 训练过程

###### 初始化

假设句子s={$A_1$,$A_2$,$A_3$}(为了对应下面的举例，先展示设置句子长度为3，也可以设更长)，现在遍历到了，特征模板先只考虑U06:%x[-1,0]/%x[0,0]/%x[1,0]，该模板生成的特征函数如下：

if(x1==’$A_1$ ’&x2==’$A_2$’&x3==’$A_3$’&y==’B’) return 1 else return 0;

if(x1==’$A_1$’&x2==’$A_2$’&x3==’$A_3$’&y==’M’) return 1 else return 0;

if(x1==’$A_1$’&x2==’$A_2$’&x3==’$A_3$’&y==’E’) return 1 else return 0;

if(x1==’$A_1$’&x2==’$A_2$’&x3==’$A_3$’&y==’O’) return 1 else return 0;

对于上面的特征函数，我们先假设所有权重$\lambda$的值均为0.5

###### 根据特征模板生成特征函数

如果需要标注的语料有N个，设置的特征模板有s个，可被标注的类别有t种，那么就会生成$N*s*t$个特征函数

例如，识别的预料为"我爱北京天安门"，设置了U00、U01、U02三个特征模板，每个文字只能被标注为B、M、E三种，那么在训练过程中，就会生成$7*3*3$，共63个特征函数

###### 学习并更新参数

然后开始遍历语料库中的每一行，每一行对U06模板都能产生4个特征函数，也会产生4个特征函数的输出值，比如遍历到“人民网1月1日”的“网”时，特征函数如下：

if(x1==’民’&x2==’网’&x3==’1’&y==’B’) return 1 else return 0;

if(x1==’民’&x2==’网’&x3==’1’&y==’M’) return 1 else return 0;

if(x1==’民’&x2==’网’&x3==’1’&y==’E’) return 1 else return 0;

if(x1==’民’&x2==’网’&x3==’1’&y==’O’) return 1 else return 0;


由于这是个有监督的学习，我们是知道真实标注的，对于“网”的上一句是“民”，下一句是“1”的条件下，“网”对应的标注是E，这是已知的。上面的四个特征函数，除了第三个输出值为1，其他的输出值为0，那么第三个对应的权重$\lambda$会从0.5增加到0.6，其他三个权重从0.5降低到0.4，这是因为语料训练过程中，语料的真实标注告诉了模型第三个规则if(x1==’A1 ’& x2==’A2 ’& x3==’A3 ’& y==’E’) return 1 else return 0是有利于正确的标注的，其他的规则是不太有利的，模型就相应地调整对应的权重，让有利于正确标注的规则能提高真实标注的分数，降低错误标注的分数。

然后窗口向下运动，遍历更多的文字，权重也在过程中不断改变。

##### 公式化

上面说的过程，可以用迭代缩放或梯度上升法。

我们较为熟悉的梯度上升法是这么做的：

一开始，为CRF模型初始化权重值，为了使这些随机初始的权重值最终调整为正确的值，遍历预处理好的训练集中的每一个训练样本（每次都遍历语料库中的每一行来更新参数)执行如下操作：

+ 遍历每一个特征函数$f_i$，并为$\lambda_i$计算训练样本的对数概率的梯度值

![Markdown](http://i2.tiimg.com/611786/edd70e37cdbafe78.png)

**梯度公式中的第一项是特征$f_i$在正确标注下的贡献，第二项是特征$f_i$在当前模型中的贡献**

+ 将$\lambda_i$朝着梯度方向移动

![Markdown](http://i2.tiimg.com/611786/3c3a081edcae4650.png)

**其中，$\alpha$是学习率**

+ 重复上述步骤，直到更新值低于阈值

#### 找到最优输出序列

经过第二步的参数学习，训练好了CRF模型，这时来了一个新的句子，如何去标注？

有一个很好的方法，动态规划算法找到最佳标注，类似于HMM中的Viterbi算法

#### 测试
这是个有监督的学习，语料库中已经有了正确的标注，最后需要将模型的结果和正确的结果进行比较，通过查看F1值查看模型的效果。


![Markdown](http://i1.fuimg.com/611786/c314ccc9189890d4.png)


## BiLSTM + crf 简化特征函数过程

使用单纯的crf处理命名实体识别问题，使用特征模板生成$N*s*t$特征函数，再根据特征函数学习参数的过程是比较繁琐复杂的，BiLSTM能够简化这个过程

假设数据集只有两个实体类型，Person（人）和Organization（组织），用B、I、O方法进行标注，这样就有五个实体标签：B-Person、I-Person、B-Organization、I-Organization、O 

设一个句子X中包含5个词，分别为$w_0、w_1、w_2、w_3、w_4$，其中，$w_0、w_1$属于Person类，$w_2$是O，$w_2$属于Organization类，$w_4$是O

$w_0、w_1、w_2、w_3、w_4$都要表示成向量的形式，先在character embedding层随机初始化，然后放进CNN中提取特征，就得到了对应的向量形式；word embedding层使用外部预训练文件中的向量；将character embedding和word embedding拼接成该单词的最终向量表示。最终向量就可以放进BiLSTM中了。

![Markdown](http://i1.fuimg.com/611786/bd29e6c376ad56e6.png)

到这里，只是BiLSTM派上了用场，CRF就可以出场了。

CRF层的加入可以对BiLSTM层的最终预测标签添加约束条件，在训练过程中，CRF层可以自动从训练数据集中学习这些约束（也就是单纯的CRF中特征函数+学习参数的过程）


### CRF层如何学习这些约束

**什么是约束**

约束也就是上面crf中说的规则，比如，句子中第一个单词的标签应为“B-”或“O”开头，“I”不能作为开头

加上这些有用的越是，可以大大减少无效预测标签序列的数量

**如何学习约束**

与单词的crf一样，大致分为训练和测试两个过程：

+ 训练：拿着预处理好的一句话（带有金标）输入神经网络中，就会预测出这句话的标注结果，我们就会用预测值和金标值来计算损失函数，然后反向传播之类的方法来计算梯度和更新参数，最后找到一个感觉很好的模型。 

+ 测试：模型训练好就是测试，相当于我们来检测这个模型的好坏了，这一步比训练少了计算损失函数这一步骤。然后使用维特比解码来找到一条最优路径，即预测标签序列

**实现的重点是损失函数的计算：**

先定义两个分数Emission score 、Transition score

**Emission score：**
 emission score来自于BiLSTM层，

![Markdown](http://i2.tiimg.com/611786/951670be988b2d38.png)

带入上图的数值，也就是：

![Markdown](http://i1.fuimg.com/611786/cdce41f8b0fc0131.png)

使用$x_{iy_i}$表示$w_i$被标注为B-Person/I-Person/B-Organization/I-Organization/O的分数

**Transition score**

使用$t_{y_iy_j}$表示标签B-Person到I-Person的转移分数，为了更好地表达，在矩阵中增加了Start和End，分别代表句子开始了和句子结束了

![Markdown](http://i2.tiimg.com/611786/6be98319018d78c6.png)

**损失函数:**

损失函数由实际路径分数$P_{real}$和所有可能的路径的总分数$P_{total}$组成。在所有可能的路径中，实际路径的得分是最高的

对于句子X来说，它的所有可能的路径是：

1.  START B-Person B-Person B-Person B-Person B-Person END 
2.  START B-Person I-Person B-Person B-Person B-Person END 
3.  ……. 
4.  START B-Person I-Person O B-Organization O END 
5.  ……. 
6.  O O O O O O O 

总共有N条可能的路径，每条路径都有一个分数$P_i$，那么所有的路径的总分数为$P_{tota}=P_1+P_2+...+P_N$

我们可以定义$P_i=e^{s_i}$，其中，$s_i$=Emission score+Transition score

对于实际路径：
![Markdown](http://i2.tiimg.com/611786/63dd840f236aec9b.png)

Emission score = $x_{0,start}+x_{1,B-Person}+x_{2,I-Person}+x_{3,0}+x_{4，B-Organization}+x_{6,End}$ = 0 + 1.5 +0.4 + 0.1 + 0.3 + 0.002 + 0 = 2.302

$x_{0,start}$和$x_{6,End}$默认设置为0

Transition score = $t_{start到B-Person}+t_{B-Person到I-Person}+t_{I-Person到O}+t_{O到B-Organization}+t_{B-Organization到O}+t_{O到end}$

如此就很容易算出$P_{real}$

计算损失函数：

$\begin{aligned}
&\text {Lossfunction}=-\ln \frac{P_{\text {Real}}}{P_{1}+P_{2}+\ldots+P_{N}}\\
&=-\ln \frac{e^{S_{\text {Real}}}}{e^{s_{1}}+e^{S_{2}}+\ldots+e^{S_{N}}}
\end{aligned}$
$\begin{array}{l}
=-\left(\ln \left(e^{S_{\text {Real}}}\right)-\ln \left(e^{S_{1}}+e^{S_{2}}+\ldots+e^{S_{N}}\right)\right) \\
=-\left(e^{S_{\text {Real}}}-\ln \left(e^{S_{1}}+e^{S_{2}}+\ldots+e^{S_{N}}\right)\right) \\
=-\left(\sum_{i=1}^{L} x_{i y_{i}}+\sum_{i=1}^{L-1} t_{y_{i} y_{i+1}}-\ln \left(e^{S_{1}}+e^{S_{2}}+\ldots+e^{S_{N}}\right)\right)
\end{array}$

**现在需要找到一个有效的方式来计算$\ln (e^{S_{1}}+e^{S_{2}}+\ldots+e^{S_{N}})$**:

举一个简单的例子来说明，给定一个句子，它的长度为 3 ， $x = [w_1,w_2,w_3]$，假设我们的数据中的标签只有 2 个，$label = {l_1,l_2}$

Emission Score:

 ![Markdown](http://i2.tiimg.com/611786/21a62307bf94d7ea.png)

Transition Score :

![Markdown](http://i2.tiimg.com/611786/2af2ba03ab530f96.png)

这一个过程是一个分数累加的过程。简言之，在这个例子中就是$w_0$的所有可能路径的总分数计算完成之后，我们再计算$w_0$到$w_1$所有可能路径的总分数，最后我们使用新的总分来计算$w_0$到$w_1$到$w_2$的总分，最终的得分就是我们需要的。 

先定义两个变量：obs 和 previous。previous 存储前面步骤的最终结果。obs 表示来自当前单词的信息。 如果在我们的句子中只有一个词$w_0$那么我们就没有来自于先前步骤的结果，那么 previous 为 None，我们只能观察到第一个词，即obs=[$x_{01}$,$x_{02}$],其中，$x_{01}$和$x_{02}$就是上边提到的 Emission Scores

$w_0$：
obs=[$x_{01}$,$x_{02}$]

Previous = None 

那么$w_0$所有可能路径的总分数是多少呢？它只有两种可能，要么被标注为$l_1$ ,要么被标注为$l_2$ ，因为只有一个词，所以不涉及转移分数的计算。

$$\begin{array}{l}
\text { TotalScore }\left(w_{0}\right) \\
=p_{1}+p_{2} \\
=e^{S_{1}}+e^{S_{2}} \\
=\ln \left(e^{x_{01}}+e^{x_{02}}\right)
\end{array}$$


${w}_{0} \rightarrow \mathrm{w}_{1}$
$$\begin{array}{l}
o b s=\left[x_{11}, x_{12}\right] \\
\text { previous }=[x_{01},x_{02}]\\
\end{array}$$

我们可以想一下，现在是两个词，标注像一个排列组合一样，两个词共有四种组合，为了方便计算我们分别把 obs 和 previous 都扩充计算，这样比较快（相当于你列举出来所有的可能一样。） 

$$\begin{array}{l}
\text { previous }=\left(\begin{array}{ll}
x_{01} & x_{01} \\
x_{02} & x_{02}
\end{array}\right) \\
\text { obs }=\left(\begin{array}{ll}
x_{11} & x_{12} \\
x_{11} & x_{12}
\end{array}\right)
\end{array}$$

把 previous, obs 和 Transition Score 相加，记作 scores：

$$\text {scores}=\left(\begin{array}{cc}
x_{01} & x_{01} \\
x_{02} & x_{02}
\end{array}\right)+\left(\begin{array}{cc}
x_{11} & x_{12} \\
x_{11} & x_{12}
\end{array}\right)+\left(\begin{array}{cc}
t_{11} & t_{12} \\
t_{21} & t_{22}
\end{array}\right)=\left(\begin{array}{cc}
x_{01}+x_{11}+t_{11} & x_{01}+x_{12}+t_{12} \\
x_{02}+x_{11}+t_{21} & x_{02}+x_{12}+t_{22}
\end{array}\right)$$

${w}_{0} \rightarrow \mathrm{w}_{1}\rightarrow \mathrm{w}_{2}$

$$\begin{array}{l}
o b s=\left[x_{21}, x_{22}\right]
\end{array}$$

$$\text {previous}=\left[\ln \left(e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}}\right), \ln \left(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}}\right)\right]$$

把 previous, obs 和 TransitionScore 相加：

$$\begin{array}{l}
\operatorname{scores}=\left(\begin{array}{ll}
\ln \left(e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}}\right) & \ln \left(e^{x_{01}+x_{11}+t_{11}}+e^{x_{20}+x_{11}+t_{21}}\right) \\
\ln \left(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}}\right) & \ln \left(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}}\right)
\end{array}\right)+\left(\begin{array}{ll}
x_{21} & x_{22} \\
x_{21} & x_{22}
\end{array}\right)+\left(\begin{array}{cc}
t_{11} & t_{12} \\
t_{21} & t_{22}
\end{array}\right) \\
=\left(\begin{array}{ll}
\ln \left(e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}}\right)+x_{21}+t_{11} & \ln \left(e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}}\right)+x_{22}+t_{12} \\
\ln \left(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}}\right)+x_{21}+t_{21} & \ln \left(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}}\right)+x_{22}+t_{22}
\end{array}\right)
\end{array}$$

为下一轮迭代更新 previous 的值: 

$$previous = [ln(e^{ln(e^{x_{01}+x_{11}+t_{11}})+e^{x_{02}+x_{11}+t_{21}})+x_{21}+t_{21}}+e^{ln(e^{x_{01}+x_{12}+t_{12}})+e^{x_{02}+x_{12}+t_{22}})+x_{21}+t_{21}})，ln(e^{ln(e^{x_{01}+x_{11}+t_{11}})+e^{x_{02}+x_{11}+t_{21}})+x_{22}+t_{12}}+e^{ln(e^{x_{01}+x_{12}+t_{12}})+e^{x_{02}+x_{12}+t_{22}})+x_{22}+t_{22}})]=[ln((e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}})e^{x_{22}+t_{11}}+(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}})e^{x_{21}+t_{21}}),ln((e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}})e^{x_{22}+t_{12}}+(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}})e^{x_{22}+t_{22}})]$$

像上一个迭代那样，我们使用新的 previous 里的元素来计算${w}_{0} \rightarrow \mathrm{w}_{1}\rightarrow \mathrm{w}_{2}$总分数。 

$$\begin{aligned}
&\text {TotalScore}\left(w_{0} \rightarrow w_{1} \rightarrow w_{2}\right)\\
&=\ln \left(e^{\text {previous}[0]}+e^{\text {previous}[1]}\right)\\
&=\ln \left(e^{ln((e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}})e^{x_{21}+t_{11}}+(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}})e^{x_{21}+t_{21}})}+e^{ln((e^{x_{01}+x_{11}+t_{11}}+e^{x_{02}+x_{11}+t_{21}})e^{x_{22}+t_{12}}+(e^{x_{01}+x_{12}+t_{12}}+e^{x_{02}+x_{12}+t_{22}})e^{x_{22}+t_{22}})}\right)\\
&=\ln \left(e^{x_{01}+x_{11}+t_{11}+x_{21}+t_{11}+} e^{x_{02}+x_{11}+t_{21}+x_{21}+t_{11}}+\ldots\right)
\end{aligned}$$

在这个句子是 3 个词，数据集中 2 个标签的例子中(总共有 8 种可能的路径)

我们实现了目标$\ln (e^{S_{1}}+e^{S_{2}}+\ldots+e^{S_{N}})$,这就是三个单词的所有可能路径的分数。又知道金标序列后就可以计算损失函数了。